# Introduction to Quantization
source:https://pytorch.org/docs/stable/quantization.html#torch.quantization.quantize_dynamic

Quantization refers to techniques for performing computations and storing tensors at lower bitwidths than floating point precision. A quantized model executes some or all of the operations on tensors with reduced precision rather than full precision (floating point) values. This allows for a more compact model representation and the use of high performance vectorized operations on many hardware platforms. PyTorch supports INT8 quantization compared to typical FP32 models allowing for a 4x reduction in the model size and a 4x reduction in memory bandwidth requirements. Hardware support for INT8 computations is typically 2 to 4 times faster compared to FP32 compute. Quantization is primarily a technique to speed up inference and only the forward pass is supported for quantized operators.

PyTorch supports multiple approaches to quantizing a deep learning model. In most cases the model is trained in FP32 and then the model is converted to INT8. In addition, PyTorch also supports quantization aware training, which models quantization errors in both the forward and backward passes using fake-quantization modules. Note that the entire computation is carried out in floating point. At the end of quantization aware training, PyTorch provides conversion functions to convert the trained model into lower precision.

At lower level, PyTorch provides a way to represent quantized tensors and perform operations with them. They can be used to directly construct models that perform all or part of the computation in lower precision. Higher-level APIs are provided that incorporate typical workflows of converting FP32 model to lower precision with minimal accuracy loss.


## Quatization API Summary

PyTorch provides three different modes of quantization: Eager Mode Quantization, FX Graph Mode Quantization (maintainence) and PyTorch 2 Export Quantization.


FX Graph Mode Quantization is an automated quantization workflow in PyTorch, and currently it’s a prototype feature, it is in maintainence mode since we have PyTorch 2 Export Quantization. It improves upon Eager Mode Quantization by adding support for functionals and automating the quantization process, although people might need to refactor the model to make the model compatible with FX Graph Mode Quantization (symbolically traceable with `torch.fx`). Note that FX Graph Mode Quantization is not expected to work on arbitrary models since the model might not be symbolically traceable, we will integrate it into domain libraries like torchvision and users will be able to quantize models similar to the ones in supported domain libraries with FX Graph Mode Quantization. For arbitrary models we’ll provide general guidelines, but to actually make it work, users might need to be familiar with `torch.fx`, especially on how to make a model symbolically traceable.

PyTorch 2 Export Quantization is the new full graph mode quantization workflow, released as prototype feature in PyTorch 2.1. With PyTorch 2, we are moving to a better solution for full program capture (torch.export) since it can capture a higher percentage (88.8% on 14K models) of models compared to torch.fx.symbolic_trace (72.7% on 14K models), the program capture solution used by FX Graph Mode Quantization. torch.export still has limitations around some python constructs and requires user involvement to support dynamism in the exported model, but overall it is an improvement over the previous program capture solution. PyTorch 2 Export Quantization is built for models captured by torch.export, with flexibility and productivity of both modeling users and backend developers in mind. The main features are (1). Programmable API for configuring how a model is quantized that can scale to many more use cases (2). Simplified UX for modeling users and backend developers since they only need to interact with a single object (Quantizer) for expressing user’s intention about how to quantize a model and what the backend support. (3). Optional reference quantized model representation that can represent quantized computation with integer operations that maps closer to actual quantized computations that happens in hardware.




The following table compares the differences between Eager Mode Quantization, FX Graph Mode Quantization and PyTorch 2 Export Quantization:


|                            | Eager Mode Quantization                                           | FX Graph Mode Quantization                                      | PyTorch 2 Export Quantization                                    |
|----------------------------|---------------------------------------------------------------------|------------------------------------------------------------------|-------------------------------------------------------------------|
| Release Status             | beta                                                                | prototype (maintenance)                                          | prototype                                                         |
| Operator Fusion            | Manual                                                              | Automatic                                                        | Automatic                                                         |
| Quant/DeQuant Placement    | Manual                                                              | Automatic                                                        | Automatic                                                         |
| Quantizing Modules         | Supported                                                           | Supported                                                        | Supported                                                         |
| Quantizing Functionals/Torch Ops | Manual                                                           | Automatic                                                        | Supported                                                         |
| Support for Customization  | Limited Support                                                     | Fully Supported                                                  | Fully Supported                                                  |
| Quantization Mode Support  | Post Training Quantization: Static, Dynamic, Weight Only           | Quantization Aware Training: Static                              | Post Training Quantization: Static, Dynamic, Weight Only         |
|                            |                                                                     |                                                                  | Quantization Aware Training: Static                              |
| Input/Output Model Type    | torch.nn.Module                                                    | torch.nn.Module (May need some refactors to make the model compatible with FX Graph Mode Quantization) | torch.fx.GraphModule (captured by torch.export)                   |


There are three types of quantization supported:

1. dynamic quantization (weights quantized with activations read/stored in floating point and quantized for compute)

2. static quantization (weights quantized, activations quantized, calibration required post training)

3. static quantization aware training (weights quantized, activations quantized, quantization numerics modeled during training)

# Eager Mode Quantization
For a general introduction to the quantization flow, including different types of quantization, please take a look at [General Quantization Flow](https://pytorch.org/docs/stable/quantization.html#general-quantization-flow).




### Post Training Dynamic Quantization
This is the simplest to apply form of quantization where the weights are quantized ahead of time but the activations are dynamically quantized during inference. This is used for situations where the model execution time is dominated by loading weights from memory rather than computing the matrix multiplications. This is true for LSTM and Transformer type models with small batch size.



PTDQ API Example: